# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #quick functions
import pandas as pd #lets you visualize tableur data
import requests #http requests
import xlsxwriter as xlsx
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'NE C EMASTK)(SA/ELG TDALBQOSRANGL',
 'calculationPrice': 'close',
 'open': 127.82,
 'openTime': 1674430614073,
 'openSource': 'lcfaifio',
 'close': 127.7,
 'closeTime': 1640099647100,
 'closeSource': 'iflafoic',
 'high': 132.7,
 'highTime': 1670449571610,
 'highSource': 'liaedeyt  reped1ci m5nu',
 'low': 128.29,
 'lowTime': 1613130361368,
 'lowSource': 'itEIri cemle arpeX ',
 'latestPrice': 130.1,
 'latestSource': 'Close',
 'latestTime': 'January 6, 2021',
 'latestUpdate': 1651608661970,
 'latestVolume': 162140223,
 'iexRealtimePrice': 131.08,
 'iexRealtimeSize': 102,
 'iexLastUpdated': 1658189718421,
 'delayedPrice': 126.94,
 'delayedPriceTime': 1682443125634,
 'oddLotDelayedPrice': 127.5,
 'oddLotDelayedPriceTime': 1671673804405,
 'extendedPrice': 129,
 'extendedChange': 1.1,
 'extendedChangePercent': 0.00912,
 'extendedPriceTime': 1633799141423,
 'previousClose': 137.07,
 'previousVolume': 102476612,
 'change': -4.

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)

In [7]:
final_dataframe.append(
    pd.Series( [symbol, price, market_cap, 'N/A'], 
               index = my_columns ),
    
    ignore_index = True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,130.1,2179079829191,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [16]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    #symbol = stock
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json() #executing http requests is one of the slowest things you can do in python
    #in the future, use batch API requests
    price = data['latestPrice']
    market_cap = data['marketCap']
    final_dataframe = final_dataframe.append(
        pd.Series( 
            [stock, price, market_cap, 'N/A'], 
               index = my_columns ),
        ignore_index = True
    )

In [17]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,124.30,38096791151,N/A
1,AAL,15.71,9708456699,N/A
2,AAP,169.65,11779688958,N/A
3,AAPL,127.60,2186373074988,N/A
4,ABBV,110.05,186755545988,N/A
...,...,...,...,...
500,YUM,108.02,33219388916,N/A
501,ZBH,167.59,33961961249,N/A
502,ZBRA,398.39,21776432702,N/A
503,ZION,49.82,8323049384,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [18]:
def chunk(lst, n):
    for i in range(0,len(lst), n):
        yield lst[i:i+n]

In [36]:
symbol_groups = list(chunk(stocks['Ticker'], 100))
symbol_strings = []
final_dataframe = pd.DataFrame(columns = my_columns)
for i in range (0, len(symbol_groups)):
    symbol_group = symbol_groups[i]
    symbol_strings.append(','.join(symbol_group))
    
for symbol_string in symbol_strings:
    batch_call_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_call_api_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series( 
                [symbol, data[symbol]['quote']['latestPrice'], data[symbol]['quote'][
                    'marketCap'], 'N/A'], 
                index = my_columns 
            ),
            ignore_index = True
        )
    
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,127.96,40435772047,N/A
1,AAL,15.37,9319165450,N/A
2,AAP,176.85,11751239832,N/A
3,AAPL,132.36,2309695163649,N/A
4,ABBV,109.02,190411795377,N/A
...,...,...,...,...
500,YUM,109.07,33623810456,N/A
501,ZBH,166.71,34489434051,N/A
502,ZBRA,411.13,22402326504,N/A
503,ZION,49.87,8344448421,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [43]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print('Invalid portfolio size. Please enter an integer.')
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)


Enter the value of your portfolio:100000
100000.0


In [45]:
#this is your position size algorithm for each stock
position_size = val/len(final_dataframe.index)
print('Your position size is: ')
print(position_size)

#algorithm for calculating the number of shares to buy
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = position_size/final_dataframe.loc[i,'Stock Price']

final_dataframe

Your position size is: 
198.01980198019803


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,127.96,40435772047,1.547513
1,AAL,15.37,9319165450,12.883526
2,AAP,176.85,11751239832,1.119705
3,AAPL,132.36,2309695163649,1.496070
4,ABBV,109.02,190411795377,1.816362
...,...,...,...,...
500,YUM,109.07,33623810456,1.815529
501,ZBH,166.71,34489434051,1.187810
502,ZBRA,411.13,22402326504,0.481648
503,ZION,49.87,8344448421,3.970720


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [58]:
#we use pandas library to intialize our writer object as opposed to xlsxwriter bc pandas is easier.
writer = pd.ExcelWriter('recommended trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [59]:
backgroundColor = '#0a0a23'
fontColor = '#ffffff'

stringFormat = writer.book.add_format(
    {
        'font_color': fontColor,
        'bg_color': backgroundColor,
        'border': 1
    }
)

dollarFormat = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': fontColor,
        'bg_color': backgroundColor,
        'border': 1
    }
)

integerFormat = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': fontColor,
        'bg_color': backgroundColor,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [60]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, stringFormat)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, stringFormat)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, stringFormat)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, stringFormat)
# writer.save()

#this overrides the headers and is just used to format the headers in the excel file
writer.sheets['Recommended Trades'].write('A1', 'Ticker', stringFormat)

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [62]:
column_formats = {
    'A': ['Ticker', stringFormat],
    'B': ['Stock Price', dollarFormat],
    'C': ['Market Capitalization', dollarFormat],
    'D': ['Number of Shares to Buy', integerFormat]
}

for column in column_formats.keys():
    #this top line fixes the formatting of the headers in the excel file
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0])
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    


## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.save()